Notre objectif est de scraper suffisamment de données sous la forme de fichiers sons (*.mp3* par exemple) pour pouvoir entraîner ensuite un ou plusieurs classificateurs de genres musicaux. 

Spotify dispose d'une API très pratique pour les développeurs, leur permettant de récupérer de nombreuses informations sur les musiques hébergées par la plateforme. Il existe également une bibliothèque Python, [spotipy](https://spotipy.readthedocs.io/en/2.16.1/), qui permet d'utiliser très facilement cette API via Python. 

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np
import time
import pickle
import random as rd
import os

Pour accéder à l'API de Spotify, il est nécessaire de posséder des identifiants clients, obtenables avec un compte Spotify. Puisque nous en possédons un, nous avons stocké les informations sur un fichier que l'on extrait ensuite pour accéder à la base de données Spotify.

In [5]:
with open('Data//spotify_cids', 'r') as id_file :
    client_id, secret_id = id_file.read().split(':')

Les deux lignes suivantes nous authentifient auprès de Spotify et nous permettent d'accéder aux données.

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = secret_id)

sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

Spotipy dispose d'une commande *search* qui permet de retourner un certain nombre d'objets (artistes, albums, chansons, podcasts, playlists...) à partir d'un terme passé en argument. Le nombre maximum d'objets retournés par requête est de 50. 

Il nous faut d'abord définir une liste de genres musicaux à scrapper. Les genres doivent être suffisamment généraux pour regrouper de nombreuses chansons, suffisamment différent entre eux, et également aisément différenciable par un humain.

Nous nous sommes finalement fixés sur 9 genres différents : 

    Rock, Jazz, Blues, Classique, Country, Hip-Hop, Disco, Metal et House/Electro
    
Notre démarche est donc la suivante : 

  1. Récupérer un grand nombre de musiques via la commande search
  2. Associer à chaque chanson son genre musical (Spotify n'associe pas un genre musical à chaque musique, mais la plupart des artistes sont associés à une liste de genres musicaux. L'astuce pour retrouver le genre d'une chanson est donc d'aller récupérer le genre "général" du créateur de la musique). 
  4. Récupérer pour chaque chanson quelques features proposées directement par Spotify (danceability, energy...), dont nous mesurerons la pertinence pour la classification
  5. Récupérer un extrait *.mp3* ou autre de quelques dizaines de secondes pour extraire nous même de nouvelles features

Un problème que nous avons rencontré est que les recherches sont généralement limitées à 2000 résultats, ce qui est insuffisant pour notre projet. Nous choisissons donc d'utiliser une liste de termes à rechercher, pour obtenir 2000 musiques différentes sur chacun de ces termes.

In [7]:
def check_genres(tracks, database, spotify_module):
    """
    La fonction prend en entrée une liste de musiques *tracks*, ainsi qu'un dictionnaire *database* 
    Elle retourne le dictionnaire *database* auquel on a ajouté les musiques de *tracks* pour lesquelles au moins 
    un genre a pu être associé
    """
    
    for track in tracks :
        
        # Récupère le dictionnaire avec les infos de l'artiste principal de la musique
        artistDict = track['artists'][0]
        
        # Récupère les genres musicaux associés à l'artiste de la musique
        # La fonction .artist retourne un dictionnaire contenant de nombreuses informations
        # ici c'est la clé "genres" qui nous intéresse
        # pour identifier l'artiste de la chanson, on utilise ici l'url spotify de la musique
        track_genres = spotify_module.artist(artistDict["external_urls"]["spotify"])["genres"]
        
        # Certains artistes n'ont aucun genre associés et la liste est donc vide
        # On ajoute le morceau à la base de données que si au moins un genre est associé à l'artiste
        if track_genres :
            
            trackInfos = {'track_name' : track['name'],
                          'track_artists' : artistDict['name'],
                          'genres' : track_genres}
            
            trackId = track['id']
            
            # On ajoute toutes les infos à notre base de données en utilisant l'id du morceau comme clé
            # Ainsi, si la chanson a déjà été retourné lors d'une autre requête, elle n'apparaitra pas en doublon dans le dictionnaire
            database[trackId] = trackInfos
            
    return database
            
        

def create_database (searchTerms, spotify_module, savename, querySize = 50):
    """
    La fonction prend en argument *searchTerms* une liste de string à utiliser pour effectuer les requêtes
    Elle prend également le module spotipy, ainsi que le nombre de chansons renvoyés à chaque requête *querySize*
    *savename* correspond au nom du fichier dans lequel le dictionnaire des données est sauvegardé
    """

    # tracksId est le dictionnaire dans lequel on va stocker toutes les informations des pistes 
    # qui nous intéressent pour la première étape du scrapping
    tracksId = dict()
    
    overall_start = time.time()
    
    # Tant que la base de données n'a pas le taille souhaitée et qu'il reste des termes, les requêtes continuent
    while searchTerms :
        
        searchTerm = searchTerms.pop(0)
        
        start_time = time.time()
        for offset in range(0, 2000, 50):
            
            # On récupère *limit* (=querySize) chanson par requête. 
            # Le paramètre type spécifie que l'on veut des objets "tracks" et pas des artistes ou des playlists par ex
            # Le paramètre offset spécifie le décalage que l'on veut dans la liste des résultats de la requête 
            # (puisqu'on ne peut avoir que 50 chansons par requête mais une requête propose en général 2000 résultats)
            queryResults = spotify_module.search(searchTerm, limit = querySize, 
                                                 type='track', offset=offset)
            
            trackList = queryResults['tracks']['items']
            
            # On utilise le résultat de la recherche pour alimenter la base de données
            tracksId = check_genres(trackList, tracksId, spotify_module = spotify_module)
            
        print('%s done in %.2f minutes' % (searchTerm, (time.time() - start_time)/60))
        print('%d tracks scrapped. Total time : %.2f\n' % (len(tracksId), (time.time() - overall_start)/60))
        
        # A chaque searchTerm, on sauvegarde les données scrappées dans un fichier pickle pour ne pas perdre le travail
        with open('%s.pickle' % savename, 'wb') as dict_file:
            pickle.dump(tracksId, dict_file, protocol=pickle.HIGHEST_PROTOCOL)    
            
    return tracksId

On définit une liste de termes à chercher qui correspond aux genres que l'on veut obtenir (certains apparaissent plus souvent car ils sont plus difficiles à obtenir, et il nous faut une base de données équilibrées).

On ajoute également certains termes génériques pour les données. 

In [8]:
searchTerms = ['Essentials', 'Best of','2010', '1990', '1980', '1970',
                'Classical', 'Mozart', 'Romance',
               'Country', 'Wild Country', 'Country Gold',
               'Rock', 'Rock and Roll',
               'Jazz', 'Jazz Vibes', 'Jazzy',
               'Hip Hop', 'Rap',
               'Blues', 'Roots', 'Acoustic Blues'
               'Disco', 'Fever', 'Dance', 'Funk',
               'Metal', 'Heavy Metal', 'Thrash Metal',
               'House', 'Electro', 'edm', 'techno']

In [9]:
# 2000 requêtes par terme correspondent à environ 3 minutes 
#tracksDict = create_database(searchTerms, sp, "Data//raw_scrap")

On dispose maintenant d'un certain nombre de chansons avec une liste de genres associés. Il faut cependant réussir à extraire de cette liste un genre global correspondant à l'un de nos 9 genres (quand c'est possible), et équilibrer la base en ajoutant le même nombre de musiques pour chaque genre

In [10]:
# On récupère éventuellement les données sauvegardés dans le fichier créé précédemment
with open('Data//raw_scrap.pickle', 'rb') as dict_file:
    tracksDict = pickle.load(dict_file)

In [11]:
genres = ['rock', 'metal', 'jazz', 'blues', 'disco', 'house', 'classical', 'hip hop', 'country']

In [12]:
# Une simple fonction qui permet de rassembler tous les sous-genres obtenus par le scraping réalisé, 
# afin de réfléchir à comment les organiser dans les 9 genres globaux que nous avons fixés
def count_subgenres (tracksDict):
    
    subgenresDict = {}
    for tracks in tracksDict.values():
        for subgenre in tracks['genres'] :
            subgenresDict[subgenre] = subgenresDict.setdefault(subgenre, 0) + 1
            
    return subgenresDict

subgenresDict = count_subgenres(tracksDict)
print(len(subgenresDict))

2903


On compte près de 3 000 sous-genres différents, qu'il va être difficile de classer. On peut cependant essayer déjà de les regrouper naïvement en vérifiant qu'il contienne un des mots de la liste *genres*, ainsi que quelques autres mots clés puisque certains genres (classique, disco) sont sous représentés.


In [13]:
genresAugmented = genres + ['romantic', 'baroque',
                            'funk', 'dance',
                            'electro', 'edm', 'techno', 'trance',
                            'roots', 'soul',
                            'bluegrass', 'americana']

In [14]:
genresToSubgenres = {genre : [] for genre in genresAugmented}

for subgenre in subgenresDict.keys():
    for genre in genresAugmented :
        if genre in subgenre :
            genresToSubgenres[genre].append(subgenre)

for genre, subgenresList in genresToSubgenres.items():
    print(genre, len(subgenresList))

rock 225
metal 214
jazz 88
blues 37
disco 11
house 69
classical 48
hip hop 108
country 35
romantic 12
baroque 6
funk 35
dance 31
electro 62
edm 17
techno 36
trance 26
roots 10
soul 30
bluegrass 7
americana 19


On obtient peut être une liste satisfaisante de sous-genres à partir de ces genres "élargis", construisons de nouveaux objets pour le vérifier

In [15]:
subgenresToGenre = {}

for genre, subgenresList in genresToSubgenres.items():
    
    for subgenre in subgenresList :
        
        if genre == 'romantic' or genre == 'baroque' :
            subgenresToGenre[subgenre] = 'classical'
            
        elif genre == 'bluegrass' or genre == 'americana' :
            subgenresToGenre[subgenre] = 'country'
            
        elif genre == 'soul' or genre == 'roots':
            subgenresToGenre[subgenre] = 'blues'
            
        elif genre == 'electro' or genre == 'edm' or genre == 'techno' or genre == 'trance' :
            subgenresToGenre[subgenre] = 'house'
        
        elif genre == 'funk' or genre == 'dance':
            subgenresToGenre[subgenre] = 'disco'
            
        else :
            subgenresToGenre[subgenre] = genre

subgenresToGenre

{'classic garage rock': 'rock',
 'classic rock': 'rock',
 'folk rock': 'rock',
 'psychedelic rock': 'rock',
 'rock': 'rock',
 'rock-and-roll': 'rock',
 'soft rock': 'rock',
 'christian alternative rock': 'rock',
 'country rock': 'country',
 'yacht rock': 'rock',
 'album rock': 'rock',
 'art rock': 'rock',
 'symphonic rock': 'rock',
 'roots rock': 'blues',
 'garage rock': 'rock',
 'modern alternative rock': 'rock',
 'modern blues rock': 'blues',
 'modern hard rock': 'rock',
 'modern rock': 'rock',
 'rockabilly': 'rock',
 'kindie rock': 'rock',
 'alternative rock': 'rock',
 'electronic rock': 'house',
 'indie rock': 'rock',
 'hard rock': 'rock',
 'rap rock': 'rock',
 'blues rock': 'blues',
 'heartland rock': 'rock',
 'turkish rock': 'rock',
 'rock steady': 'rock',
 'alternative roots rock': 'blues',
 'british indie rock': 'rock',
 'dance rock': 'disco',
 'british alternative rock': 'rock',
 'welsh rock': 'rock',
 'german rock': 'rock',
 'pub rock': 'rock',
 'southern rock': 'rock',
 'pop

On obtient un dictionnaire où chaque sous-genre renvoie vers son genre parent

Rassemblons maintenant les chansons par genre musical

In [16]:
tracksByGenre = {}

for trackId, trackInfo in tracksDict.items():
    
    trackGenre = ''
    
    # On parcourt les sous genres associés au morceau
    for trackSubgenres in trackInfo['genres'] :
        # Si un sous genre est présent dans le dictionnaire rassemblant les sous genres qui nous intéressent
        if subgenresToGenre.get(trackSubgenres) :
            # On assigne ce genre au morceau et on arrête de parcourir les sous genres associés au morceau
            trackGenre = subgenresToGenre.get(trackSubgenres)
            break
            
    # Si l'on a assigné un genre au morceau, alors on l'ajoute à notre nouveau dictionnaire avec les infos correspondantes
    if trackGenre :
        tracksByGenre[trackId] = {'track_name' : trackInfo['track_name'],
                                  'track_artists' : trackInfo['track_artists'],
                                  'genre' : trackGenre}
            
        
    

On formate ensuite nos données sous forme de dataframe pandas

In [17]:
tracksDatabase = pd.DataFrame(tracksByGenre).transpose()

tracksDatabase.genre.value_counts()

rock         9028
classical    4651
country      3294
disco        3255
hip hop      3178
metal        2356
jazz         2163
blues        2065
house        1755
Name: genre, dtype: int64

On observe que même en boostant quelques genres musicaux (notamment electro-house) il est difficile d'augmenter leur nombre. On obtient tout de même au moins 1750 morceaux de chaque genre, ce qui peut nous permettre de constituer une base de données de plus de 15000 musiques différentes à classer. Cela nous semble raisonnable pour continuer.

In [18]:
def balance_database (database, numPerClass, genreList):
    
    balancedDb = pd.DataFrame()
    
    for genre in genreList :
        
        filteredIndex = rd.sample(list(database[database.genre == genre].index), k = numPerClass)
        balancedGenre = database.loc[filteredIndex]
        
        balancedDb = pd.concat([balancedDb, balancedGenre], axis = 0)
        
    return balancedDb

La fonction prend en argument la base de données déséquilibrées, le nombre de musiques que l'on veut par classe et le nom de chaque classe (les genres musicaux) et retourne une base de données équilibrée constituée aléatoirement 

In [19]:
tracksDatabaseBalanced = balance_database(tracksDatabase, 
                                          numPerClass = tracksDatabase.genre.value_counts()[-1],
                                          genreList = genres)

# On vérifie bien que la base est équilibrée
print(len(tracksDatabaseBalanced))

print("\nNb of tracks per genres :")
print(tracksDatabaseBalanced.genre.value_counts())

print("\nAll tracks are represented only once :")
print(tracksDatabaseBalanced.index.value_counts())

15795

Nb of tracks per genres :
disco        1755
hip hop      1755
rock         1755
metal        1755
country      1755
house        1755
jazz         1755
classical    1755
blues        1755
Name: genre, dtype: int64

All tracks are represented only once :
3RrumTYvxSLX8SwLrZCjNd    1
1TrGdXSgiBm8W68D2K1COG    1
6ZRnVnjp57UMZPQqVLMZgS    1
6DHrIQHwu9GRrqzocQvONi    1
2lCY26XhIuoDGJolzRIyd3    1
                         ..
1wGYcpJZ59rePkNu4zHmLu    1
5zIt9z09XeV59kTyZ6tTbq    1
7CsQdepFbZSNrisMKeZffg    1
4ARyqqJXPiHpqVbLtF6XEb    1
6kyV55FlbuERNL3YK2RivS    1
Length: 15795, dtype: int64


On a donc réussi à rassembler les indentifiants de 1755 chansons différentes pour chacun de nos 9 genres, en élargissant un peu certaines familles (house/electro/techno ou disco/funk par exemple).

Il nous faut maintenant récupérer les features et les extraits de chacun de ces chansons.

In [22]:
def get_spotify_features (tracksDb, spotify_module, 
                          savename = 'Data//spotify_features.csv',
                          dropFields = ['type', 'uri', 'track_href', 'analysis_url']):
    
    # On crée un dictionnaire avec pour index les identifiants des chansons
    # Et pour valeur le dictionnaire correspondant qui contient le nom de la musique, l'artiste et le genre associés
    featuresDict = {trackId : dict(tracksDb.loc[trackId]) for trackId in tracksDb.index}
    
    for trackId, trackInfos in featuresDict.items():
        
        # Pour chaque identifiant, on récupère les features spotify
        track_features = spotify_module.audio_features(trackId)[0]
        
        # Parfois, mais très rarement, la liste des features est vide. 
        # On ne l'ajoute à la base que si la liste n'est pas vide
        if track_features :
            trackInfos.update(track_features)
            
    # On transforme le dictionnaire en dataframae
    features_df = pd.DataFrame(featuresDict).transpose().drop(dropFields, axis = 1)
    features_df.to_csv('%s.csv' % savename, index = False)
    
    return features_df
            
        

In [30]:
#spotify_features = get_spotify_features(tracksDatabaseBalanced, sp)

L'API de spotify permet également de récupérer des éléments "d'analyse" effectués par spotify. On retrouve par exemple la durée estimée des mesures ou le nombre de battements par minute (une estimation du tempo). Ce qui nous intéresse, ce sont la liste des estimations des hauteurs des notes au cours du temps (leur emplacement dans la gamme) ainsi que leur timbre respectif (leur "aspect" musical). 

Un problème est que, puisque le tempo et la durée des musiques est différentes, la longueur des listes contenant les timbres et les hauteurs sont différentes. Pour pallier cela, nous avons choisi de ne récupérer les informations que des 1500 premières notes. Suivant le tempo de la musique et les notes jouées, les 1500 premières notes correspondent à environ 25 à 50 secondes au début de la chanson. On peut imaginer cette durée largement suffisante pour estimer le genre d'une musique, l'oreille humaine y parvenant généralement en quelques secondes. 

In [31]:
def get_spotify_analysis (tracksDb, spotify_module, 
                          savedNotes = 1500,
                          savename = 'Data//spotify_analysis'):
    
    # On crée deux dictionnaires qui contiendront les listes des hauteurs ou des timbres pour chaque piste
    pitchesDict = {}
    timbresDict = {}
    
    # Pour chaque piste, on récupère l'analyse de spotify
    for trackId in tracksDb.index:
        
        # On ajoute une exception car parfois la recherche de l'analyse provoque une erreur HTTP sans que l'on comprennne pourquoi
        # Probablement parce que l'analyse n'est pas disponible pour cette piste
        try : 
            trackAnalysis = spotify_module.audio_analysis(trackId)
            
            # On ne s'intéresse qu'aux savedNotes/12 premiers segments, car les segments sont rassemblés en groupe de 12 notes
            # On ajoute les hauteurs et les timbres des premières notes si et seulement si la longueur est suffisante. 
            if len(trackAnalysis['segments']) >= savedNotes//12 :
                pitchesDict[trackId] = []
                timbresDict[trackId] = []
            
                for segment in trackAnalysis['segments'][:savedNotes//12]:
                    pitchesDict[trackId].extend(segment['pitches'])
                    timbresDict[trackId].extend(segment['timbre'])
        except :
            print('HTTP Error for : ', trackId)
            
        # On sauvegarde ensuite le dictionnaire à chaque étape, car les calculs sont longs et les erreurs fréquentes
        
        with open('%s_pitches.pickle' % savename, 'wb') as dict_file:
            pickle.dump(pitchesDict, dict_file, protocol=pickle.HIGHEST_PROTOCOL)
            
        with open('%s_timbres.pickle' % savename, 'wb') as dict_file:
            pickle.dump(timbresDict, dict_file, protocol=pickle.HIGHEST_PROTOCOL)  
            
    pitches_df = pd.DataFrame(pitchesDict).transpose()
    timbres_df = pd.DataFrame(timbresDict).transpose()
    
    pitches_df.to_csv('%s_pitches.csv' % savename, index = False)
    timbres_df.to_csv('%s_timbres.csv' % savename, index = False)
    
    return pitches_df, timbres_df
        
    

In [32]:
# pitches_df, timbres_df = get_spotify_analysis(tracksDatabaseBalanced, sp)

HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/54T5ibGnwfdM5ez7xqwJQl returned 404 due to analysis not found


HTTP Error for :  54T5ibGnwfdM5ez7xqwJQl


HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/0M2hkIRjThYwmhgPYBBVWg returned 404 due to analysis not found


HTTP Error for :  0M2hkIRjThYwmhgPYBBVWg


Max Retries reached


HTTP Error for :  3XhM0bJQzMH0fWmGepafYA


Max Retries reached


HTTP Error for :  6N3BEQOuCDf701cMMMfKMn
HTTP Error for :  6XAlmnkpNnwDJK4rRlAi1f
HTTP Error for :  3NMaN8bp1QlzVXJCF22Fgv


HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/4YW7G6KNkM9JoB7SLB3MZM returned 404 due to analysis not found


HTTP Error for :  4YW7G6KNkM9JoB7SLB3MZM


On constate quelques erreurs mais dans l'ensemble on a réussi à récupérer les données pour la plupart des pistes

In [46]:
print("Nombre de pistes dont on a pu obtenir les données d'analyses :", len(pitches_df))
print('Si in seul index différe, la somme sera différente de 0 :', sum(pitches_df.index != timbres_df.index))

Nombre de pistes dont on a pu obtenir les données d'analyses : 15741
Si in seul index différe, la somme sera différente de 0 : 0


On voit bien que les index sont identiques, et on dispose au total de 15741 morceaux différents.

On ajoute simplement les labels aux deux dataframe timbres et hauteurs :

In [55]:
for trackIdx in pitches_df.index :
    trackGenre = tracksDatabaseBalanced.loc[trackIdx, 'genre']
    pitches_df.loc[trackIdx, 'genre'] = trackGenre
    timbres_df.loc[trackIdx, 'genre'] = trackGenre

On va rééquilibrer la base de données une dernière fois, puisque les données d'analyses n'étaient pas disponibles pour certaines pistes :

In [63]:
pitches_df.genre.value_counts()

disco        1755
hip hop      1755
country      1755
blues        1755
rock         1753
house        1753
jazz         1753
metal        1751
classical    1711
Name: genre, dtype: int64

In [92]:
pitches_df_final = balance_database(pitches_df, 
                                    numPerClass = pitches_df.genre.value_counts()[-1], 
                                    genreList = genres)

Les données sont bien équilibrées :

In [98]:
pitches_df_final.genre.value_counts()

disco        1711
hip hop      1711
rock         1711
metal        1711
country      1711
house        1711
jazz         1711
classical    1711
blues        1711
Name: genre, dtype: int64

On ajuste les deux autres dataframe en accord avec celui contenant les hauteurs des notes

In [104]:
timbres_df_final = timbres_df[timbres_df.index.isin(pitches_df_final.index)]
tracksDatabaseFinal = tracksDatabaseBalanced[tracksDatabaseBalanced.index.isin(pitches_df_final.index)]

On sauvegarde enfin les bases terminées :

In [105]:
pitches_df_final.to_csv("Data//tracksPitches.csv")
timbres_df_final.to_csv("Data//tracksTimbres.csv")
tracksDatabaseFinal.to_csv("Data//tracksMeta.csv")